<center><h1>Kaggle

In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import plotly.graph_objects as go
# import plotly.express as px
from sklearn.metrics import mean_absolute_percentage_error as mape # usage : print(mape(pred, observed))

import datetime

path = 'C:/Users/sebir/Desktop/M2 Toulouse/Défi IA/'

# Import data

In [2]:
# Training set, X
X = pd.read_csv(path + 'X_station_train.csv',
                parse_dates=['date'],
                infer_datetime_format=True)
X.columns = ['number_sta', 'date', 'wind_speed', 'temperature', 'dew_point_temperature', 'humidity', 'wind_direction', 'precipitation', 'Id']

X_agg_filledNA = pd.read_csv(path + 'our_X_agg_filled_nearestNeighbours.csv',
                             parse_dates=['day'],
                             infer_datetime_format=True)

# Training set, Y
Y = pd.read_csv(path + 'Y_train.csv',
                parse_dates=['date'],
                infer_datetime_format=True)

Y_filledwithNA = pd.read_csv(path + 'Y_train.csv',
                parse_dates=['date'],
                infer_datetime_format=True)

# Preprocess data

## `X` hourly to `X_agg`

We have to group the data by data in order to have something comparable with `Y`

In [3]:
X['day'] = X['date'].apply(lambda x: x.date())

In [4]:
X_agg = X.copy()
X_agg = X_agg.groupby(by = ['day', 'number_sta'], as_index = False).agg({
    "number_sta"            : "first",
    "wind_speed"            : "median",
    "temperature"           : "median",
    "dew_point_temperature" : "median",
    "humidity"              : "median",
    "wind_direction"        : "median",
    "precipitation"         : "sum"
})

We firstly drop all the NAs.

In [5]:
X_agg.dropna(inplace = True)
Y = Y.loc[X_agg.index]

Y.dropna(inplace = True)
X_agg = X_agg.loc[Y.index]

Adding the month before deleting the date, since we have this information in `X_test`:

In [6]:
X_agg['month'] = X_agg['day'].apply(lambda x: x.month)

We then delete the Id in `Y` which won't be helpful

In [7]:
X_agg.drop(['day'], axis = 1, inplace = True)
Y.drop(['date', 'Id'], axis = 1, inplace = True)

## `X_agg_filledNA`
We took `X_agg` and we filled the gaps with data from nearests neighbours. Preprocessing data for coherence with `Y`:

Adding the month before deleting the date, since we have this information in `X_test`:

In [8]:
X_agg_filledNA['month'] = X_agg_filledNA['day'].apply(lambda x: x.month)
X_agg_filledNA.drop('day', axis = 1, inplace = True)

In [9]:
Y_filledwithNA.dropna(inplace = True)
X_agg_filledNA = X_agg_filledNA.loc[Y_filledwithNA.index]

## `our_X_filled_nearestNeighbours` to `X_filled_agg`

In [26]:
X_filled = pd.read_csv(path + 'our_X_filled_nearestNeighbours.csv',
                       parse_dates=['date'],
                       infer_datetime_format=True)

In [28]:
Y_filled = pd.read_csv(path + 'Y_train.csv',
                       parse_dates=['date'],
                       infer_datetime_format=True)

In [27]:
X_filled['day'] = X_filled['date'].apply(lambda x: x.date())

X_filled_agg = X_filled.copy()
X_filled_agg = X_filled_agg.groupby(by = ['day', 'number_sta'], as_index = False).agg({
    "number_sta"            : "first",
    "wind_speed"            : "median",
    "temperature"           : "median",
    "dew_point_temperature" : "median",
    "humidity"              : "median",
    "wind_direction"        : "median",
    "precipitation"         : "sum"
})

In [29]:
X_filled_agg.dropna(inplace = True)
Y_filled = Y_filled.loc[X_filled_agg.index]

Y_filled.dropna(inplace = True)
X_filled_agg = X_filled_agg.loc[Y_filled.index]

In [30]:
X_filled_agg['month'] = X_filled_agg['day'].apply(lambda x: x.month)

X_filled_agg.drop(['day'], axis = 1, inplace = True)
Y_filled.drop(['date', 'Id'], axis = 1, inplace = True)

# Trial with Random Forest

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

In [22]:
sc = StandardScaler(with_mean=True)

X_agg_scale = sc.fit_transform(X_agg)
# X_test = sc.transform(X_test)

sc = StandardScaler(with_mean=True)

X_agg_filledNA_scale = sc.fit_transform(X_agg_filledNA)


Trying to drop all nas (84943 rows available):

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X_agg_scale, Y['Ground_truth'], test_size = 0.3)
rf = RandomForestRegressor(n_estimators = 200)
rf.fit(X_train, y_train)
y_predict = rf.predict(X_test)
print(f"Score with dropping NAs: {mape(y_predict, y_test)}")

Score with dropping NAs: 1.2088862054578846


Trying with filling the NAs with nearest neighbour's values (183747 rows):

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_agg_filledNA_scale, Y_filledwithNA['Ground_truth'], test_size = 0.3)
rf = RandomForestRegressor(n_estimators = 200)
rf.fit(X_train, y_train)
y_predict = rf.predict(X_test)
print(f"Score with dropping NAs: {mape(y_predict, y_test)}")

Score with dropping NAs: 1.118526846948131


Trying with aggregating after filling the NAs with nearest neighbour's values (162107 rows):

In [32]:
sc = StandardScaler(with_mean=True)

X_filled_agg_scale = sc.fit_transform(X_filled_agg)

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X_filled_agg, Y_filled['Ground_truth'], test_size = 0.3)
rf = RandomForestRegressor(n_estimators = 200)
rf.fit(X_train, y_train)
y_predict = rf.predict(X_test)
print(f"Score with dropping NAs: {mape(y_predict, y_test)}")

Score with dropping NAs: 37041511956.97175


## Trying to improve the RandomForest

We can try to first perform a scaler before doing the data:

In [ ]:
from sklearn.preprocessing import scale
X_scaled = scale(X_agg.drop('number_sta', axis = 1)) # centralisation et normalisation
X_scaled = pd.DataFrame(X_scaled, columns = X_agg.drop('number_sta', axis = 1).columns)
X_scaled['number_sta'] = X_agg['number_sta'].to_list()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y['Ground_truth'], test_size = 0.3)
rf_scaled = RandomForestRegressor(n_estimators = 200)
rf_scaled.fit(X_train, y_train)
y_predict = rf_scaled.predict(X_test)
print(f"Score: {mape(y_predict, y_test)}")

We can try to fit and learn several times with different parameters to see if it gives better results. Things to try:
* whether scaling gives better result or not (perform a few ones for scaled and non-scaled data, then see distribution). We can also fit the models on the same scaled and non-scaled dataset, using `random_state = xx` of `train_test_split`, to compare.
* the importance of `n_estimators`

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import scale

from tqdm.notebook import trange

# Performing the scaling once
X_scaled = scale(X_agg.drop('number_sta', axis = 1)) # centralisation et normalisation
X_scaled = pd.DataFrame(X_scaled, columns = X_agg.drop('number_sta', axis = 1).columns)
X_scaled['number_sta'] = X_agg['number_sta'].to_list()


nbTrials = 15

# Without scaling
scores_without_scaling = []
for i in trange(nbTrials):
    X_train, X_test, y_train, y_test = train_test_split(X_agg, Y['Ground_truth'], test_size = 0.3)
    rf = RandomForestRegressor()
    rf.fit(X_train, y_train)
    y_predict = rf.predict(X_test)
    scores_without_scaling.append(mape(y_predict, y_test))

# With scaling
scores_with_scaling = []
for i in trange(nbTrials):
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y['Ground_truth'], test_size = 0.3)
    rf_scaled = RandomForestRegressor()
    rf_scaled.fit(X_train, y_train)
    y_predict = rf_scaled.predict(X_test)
    scores_with_scaling.append(mape(y_predict, y_test))

In [ ]:
print("Without scaler:")
print(f"{(np.array(scores_without_scaling) < 10).sum()}/15 converged")
print(f"Mean of MAPE score = {np.array(scores_without_scaling)[np.array(scores_without_scaling) < 10].mean()}")

print("With scaler:")
print(f"{(np.array(scores_with_scaling) < 10).sum()}/15 converged")
print(f"Mean of MAPE score = {np.array(scores_with_scaling)[np.array(scores_with_scaling) < 10].mean()}")

# print("Mean with scaler:")
# np.array(scores_without_scaling)[np.array(scores_without_scaling) < 10].mean()

In [ ]:
from tqdm.notebook import tqdm

# Performing the scaling once
X_scaled = scale(X_agg.drop('number_sta', axis = 1)) # centralisation et normalisation
X_scaled = pd.DataFrame(X_scaled, columns = X_agg.drop('number_sta', axis = 1).columns)
X_scaled['number_sta'] = X_agg['number_sta'].to_list()


n_estimators = [100, 112, 125, 137, 150, 162, 175, 187, 200, 212, 225, 237, 250, 262, 275, 287, 300]

# With scaling
scores_estimators = []
for n_estimator in tqdm(n_estimators):
    print(f"With {n_estimator} trees:")
    mean = []
        
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y['Ground_truth'], test_size = 0.3)
    rf_scaled = RandomForestRegressor(n_estimators = n_estimator)
    rf_scaled.fit(X_train, y_train)
    y_predict = rf_scaled.predict(X_test)
    score = mape(y_predict, y_test)

    while score > 1000:
        print("Score was > 1000")
        # If diverged, run again until we get convergence
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y['Ground_truth'], test_size = 0.3)
        rf_scaled = RandomForestRegressor(n_estimators = n_estimator)
        rf_scaled.fit(X_train, y_train)
        y_predict = rf_scaled.predict(X_test)
        score = mape(y_predict, y_test)
            
    print("Converged")
    
    scores_estimators.append(score)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(n_estimators, scores_estimators)

# Submission

In [13]:
X_test = pd.read_csv(path + 'X_station_test.csv')
X_test

,dd,hu,td,t,ff,precip,month,Id
0,NaN,NaN,NaN,278.35,NaN,NaN,12,14047002_277_4
1,NaN,NaN,NaN,278.40,NaN,0.0,12,14047002_277_5
2,NaN,NaN,NaN,279.01,NaN,0.0,12,14047002_277_6
3,NaN,NaN,NaN,279.66,NaN,0.0,12,14047002_277_7
4,NaN,NaN,NaN,279.99,NaN,0.0,12,14047002_277_8
...,...,...,...,...,...,...,...,...
2304797,190.0,82.8,277.00,279.74,10.62,0.0,12,95690001_176_19
2304798,195.0,84.2,277.44,279.93,11.86,0.0,12,95690001_176_20
2304799,199.0,85.7,277.95,280.21,11.77,0.0,12,95690001_176_21
2304800,198.0,85.3,278.25,280.58,10.16,0.0,12,95690001_176_22
